# CHAPTER 10 실습: 마케팅 추천 엔진

『MLOps 도입 가이드』 10장을 바탕으로 **간단한 추천 시스템 파이프라인**을 실습합니다.

## 실습 목표
- 합성(user–item) 상호작용 데이터 생성
- 베이스라인(인기 기반) & 아이템 기반 CF 추천
- 오프라인 평가 (Hit@K, NDCG@K)
- MLflow로 실험 추적 (파라미터/지표/아티팩트)

⚙️ 필요 패키지: `numpy`, `pandas`, `scipy`, `scikit-learn`, `mlflow`
> 설치 예: `pip install numpy pandas scipy scikit-learn mlflow`


## 1. 데이터 생성 (합성 상호작용)

In [7]:
import numpy as np
import pandas as pd
rng = np.random.default_rng(42)

# 구성: 사용자 300명, 아이템 400개, 상호작용 15,000건
n_users, n_items, n_interactions = 300, 400, 15000
users = rng.integers(0, n_users, size=n_interactions)
items = rng.integers(0, n_items, size=n_interactions)
# 시간 스탬프 (시뮬레이션용): 0 ~ 9999
timestamps = rng.integers(0, 10000, size=n_interactions)
df = pd.DataFrame({'user': users, 'item': items, 'ts': timestamps}).drop_duplicates(['user','item','ts'])
df = df.sort_values(['user','ts']).reset_index(drop=True)
df.head()

,user,item,ts
0,0,31,182
1,0,395,279
2,0,359,540
3,0,82,878
4,0,160,912


## 2. Train/Test 분할 (Leave-One-Out by user)

In [8]:
test_idx = df.groupby('user')['ts'].idxmax()  # 각 사용자 최신 상호작용 1건
test = df.loc[test_idx].reset_index(drop=True)
train = df.drop(index=test_idx).reset_index(drop=True)
print(f"Train: {len(train)}, Test: {len(test)} (users={train['user'].nunique()})")
train.head()

Train: 14700, Test: 300 (users=300)


,user,item,ts
0,0,31,182
1,0,395,279
2,0,359,540
3,0,82,878
4,0,160,912


## 3. 평가 지표 (Hit@K, NDCG@K)

In [9]:
from math import log2
def evaluate_ranking(recommend_func, K=10):
    hits, ndcgs, evaluated = 0, 0.0, 0
    user_train_items = train.groupby('user')['item'].apply(set).to_dict()
    for _, row in test.iterrows():
        u, true_item = int(row['user']), int(row['item'])
        seen = user_train_items.get(u, set())
        recs = recommend_func(u, seen=seen, K=K)
        if not recs:
            continue
        evaluated += 1
        if true_item in recs:
            hits += 1
            rank = recs.index(true_item) + 1
            ndcgs += 1.0 / log2(rank + 1)
    hit_rate = hits / max(1, evaluated)
    ndcg = ndcgs / max(1, evaluated)
    return {'hit@K': hit_rate, 'ndcg@K': ndcg, 'evaluated': evaluated}

print('지표 함수 준비 완료')

지표 함수 준비 완료


## 4. 베이스라인: 인기(POP) 추천

In [10]:
item_pop = train['item'].value_counts().index.tolist()
def recommend_pop(user_id, seen=None, K=10):
    seen = seen or set()
    recs = [it for it in item_pop if it not in seen]
    return recs[:K]

metrics_pop = evaluate_ranking(recommend_pop, K=10)
metrics_pop

{'hit@K': 0.03, 'ndcg@K': 0.014279611271625203, 'evaluated': 300}

## 5. 아이템 기반 협업 필터링 (Cosine)

In [11]:
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

# 사용자-아이템 행렬 (train 기준)
ui = train.copy()
ui['val'] = 1.0
mat = csr_matrix(
    (ui['val'], (ui['user'], ui['item'])), shape=(n_users, n_items)
)

# 아이템-아이템 유사도 (cosine): 메모리 고려해서 400x400은 직접 계산
item_sim = cosine_similarity(mat.T, dense_output=False)  # (n_items, n_items)

item_pop_set = set(item_pop)
def recommend_itemcf(user_id, seen=None, K=10):
    seen = seen or set()
    # 사용자의 학습 아이템 벡터
    user_vec = mat[user_id]  # (1, n_items)
    # 유사도 기반 점수: S * u
    scores = item_sim.dot(user_vec.T).toarray().ravel()
    # 본 아이템 제외
    scores[list(seen)] = -1e9
    # 상위 K 추출
    top_idx = np.argpartition(-scores, range(min(K*10, len(scores))))[:max(K*2, K)]
    top_sorted = top_idx[np.argsort(-scores[top_idx])]
    recs = [int(i) for i in top_sorted if i not in seen][:K]
    # 후보 부족 시 인기 아이템로 보충
    if len(recs) < K:
        for it in item_pop:
            if it not in seen and it not in recs:
                recs.append(int(it))
                if len(recs) >= K:
                    break
    return recs

metrics_itemcf = evaluate_ranking(recommend_itemcf, K=10)
metrics_itemcf

{'hit@K': 0.04666666666666667,
 'ndcg@K': 0.021186167176967405,
 'evaluated': 300}

## 6. MLflow로 실험 추적

In [12]:
import mlflow
mlflow.set_experiment('reco_ch10')
with mlflow.start_run(run_name='pop_vs_itemcf'):
    mlflow.log_param('n_users', n_users)
    mlflow.log_param('n_items', n_items)
    mlflow.log_param('n_interactions', n_interactions)
    mlflow.log_param('K', 10)

    mlflow.log_metric('pop_hit_at_10', float(metrics_pop['hit@K']))
    mlflow.log_metric('pop_ndcg_at_10', float(metrics_pop['ndcg@K']))
    mlflow.log_metric('itemcf_hit_at_10', float(metrics_itemcf['hit@K']))
    mlflow.log_metric('itemcf_ndcg_at_10', float(metrics_itemcf['ndcg@K']))
    # 아티팩트 저장: 간단 리포트
    report = {
        'metrics_pop': metrics_pop,
        'metrics_itemcf': metrics_itemcf
    }
    mlflow.log_dict(report, 'report.json')
print('✅ MLflow 로그 완료 (mlruns/ 확인)')

✅ MLflow 로그 완료 (mlruns/ 확인)


## 7. 사용자별 추천 예시

In [ ]:
sample_user = int(rng.integers(0, n_users))
seen = set(train[train['user']==sample_user]['item'].tolist())
print('샘플 사용자:', sample_user, '| 학습에서 본 아이템 수:', len(seen))
print('POP 추천:', recommend_pop(sample_user, seen=seen, K=10))
print('ItemCF 추천:', recommend_itemcf(sample_user, seen=seen, K=10))

### 끝!
- 이 노트북은 **배치 오프라인 평가** 중심입니다.
- 온라인 서빙/실험(A/B)은 FastAPI + Vector DB/ANN(예: FAISS)로 확장할 수 있습니다.
- 원하시면 실시간 후보생성(ANN) 및 재순위화(학습된 랭커) 템플릿도 만들어 드릴게요.